In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
  f.write('{"username":"username","key":"yourkey"}')  #  Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73248 sha256=6a18aba7eae478ded870bf91822922e2ae9d55615963d64f9a6adc867d37ebde
  Stored in directory: /root/.cache/pip/wheels/b5/23/bd/d33cbf399584fa44fa049711892d333954a50ed4b86948109e
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.7.4.2
    Uninstalling kaggle-1.7.4.2:
      Successfully uninstalled kaggle-1.7.4.2


In [ ]:
!mkdir '/content/data'

!kaggle competitions download -c dna-intro-to-neural-nets-aims-rwanda-2025
!unzip -qo 'dna-intro-to-neural-nets-aims-rwanda-2025.zip' -d '/content/data'

  0% 0.00/347k [00:00<?, ?B/s]
100% 347k/347k [00:00<00:00, 41.0MB/s]


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Read the data csv and design a data loader of your choice to put the data in batches

train_dataset = pd.read_csv('/content/data/Xtr.csv', sep = ' ')
test_dataset = pd.read_csv('/content/data/Xte.csv')
train_labels = pd.read_csv('/content/data/Ytr.csv')

len(train_dataset)

2000

In [ ]:
def encode_dna_sequence(sequence):
    # Define the encoding for each nucleotide
    encoding = {
        'A': (1, 0, 0, 0),
        'C': (0, 1, 0, 0),
        'G': (0, 0, 1, 0),
        'T': (0, 0, 0, 1)
    }

    # Initialize an empty list to hold the encoded sequence
    encoded_sequence = []

    # Iterate through each nucleotide in the sequence
    for nucleotide in sequence:
        if nucleotide in encoding:
            encoded_sequence.extend(encoding[nucleotide])

    return np.array(encoded_sequence)

In [ ]:
import csv
import pandas as pd
import numpy as np

# Read the labels from the CSV file
labels_file = '/content/data/Ytr.csv'  # Replace with your actual labels file name
labels_dict = {}

with open(labels_file, mode='r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip header
    for row in reader:
        id = int(row[0])  # Read ID
        label = row[1]     # Read label
        labels_dict[id] = label

# Read the dataset from a CSV file
input_file = '/content/data/Xtr.csv'  # Replace with your actual input file name
encoded_dataset = []

with open(input_file, mode='r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip header
    for row in reader:
        id = int(row[0])  # Read ID
        seq = row[1]      # Read sequence
        encoded_seq = encode_dna_sequence(seq)
        #print(encoded_seq)
        # Check if the ID exists in the labels and combine
        if id in labels_dict:
            encoded_dataset.append((encoded_seq, labels_dict[id]))
    print(len(seq))
    print(len(encoded_seq))

# Save the combined dataset to a new CSV file
output_file = 'encoded_dataset.csv'
with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write header
    writer.writerow(["Encoded_Seq", "Label"])
    # Write combined data
    for  encoded_seq, label in encoded_dataset:
        #print(len(encoded_seq))
        writer.writerow([encoded_seq, label])  # Write encoded sequence first then the label

print(f"Combined dataset saved to {output_file}.")

101
404
Combined dataset saved to encoded_dataset.csv.


In [ ]:
import pandas as pd

train_encoded = pd.read_csv('encoded_dataset.csv', sep = ',')
train_encoded.head()

,Encoded_Seq,Label
0,[0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1...,1
1,[0 1 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0...,0
2,[0 0 1 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0...,1
3,[0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 0...,0
4,[0 0 1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0...,1


In [ ]:
sequences = train_encoded['Encoded_Seq']
labels = train_encoded['Label']
print(sequences[60])
sequences.shape

[1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 0 1
 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1
 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 0 0
 0 1 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0
 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0
 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 1 0
 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0
 1 1 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0
 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0
 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 1 0
 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0]


(2000,)

In [ ]:
train_encoded.head()

,Encoded_Seq,Label
0,[0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1...,1
1,[0 1 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0...,0
2,[0 0 1 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0...,1
3,[0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 0...,0
4,[0 0 1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0...,1


In [ ]:
import torch

train_loader = torch.utils.data.DataLoader(train_encoded, batch_size=16, shuffle=True)

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
import ast  # Import ast to evaluate string representation of lists

# Extract sequences and labels
# Convert sequences to a list
# sequences = train_encoded['Encoded_Seq'].tolist()  # This contains NumPy arrays
# labels = np.array(train_encoded['Label'])

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.2)

# Convert to PyTorch tensors
# Pad sequences to ensure they have the same length

# Convert string representations of lists to actual lists
X_train = [ast.literal_eval(seq.replace(' ',',')) for seq in X_train] # This line converts strings to lists
X_test = [ast.literal_eval(seq.replace(' ',',')) for seq in X_test] # This line converts strings to lists

#Convert to PyTorch tensors
X_train_tensor = pad_sequence([torch.tensor(seq, dtype=torch.float32) for seq in X_train], batch_first=True, padding_value=0.0)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = pad_sequence([torch.tensor(seq, dtype=torch.float32) for seq in X_test], batch_first=True, padding_value=0.0)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long) # Convert y_test Series to a NumPy array using .values before creating the tensor

# Create DataLoaders for both train and test sets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)


# Example of accessing a sample
print("Train Sample:", X_train[0])
print("Label:", y_train[0])

Train Sample: [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1

In [ ]:
import torch.nn as nn

# Set your maximum sequence length (for example, based on your dataset)
max_length = X_train_tensor.size(1)  # Determine max length dynamically based on input

class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLPModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # Input size should match max_length
        self.fc2 = nn.Linear(hidden_size, output_size)


    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input if necessary
        x = torch.relu(self.fc1(x))  # Apply ReLU activation
        x = self.fc2(x)  # Output layer
        return x



In [ ]:
def train(model, optimizer, train_loader, num_epochs=5):
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0

        for sequences, labels in train_loader:
            # Transfer data to device
            sequences, labels = sequences.to(device), labels.to(device)

            # Forward pass
            outputs = model(sequences)
            loss = criterion(outputs, labels)

            # Backpropagation
            optimizer.zero_grad()  # Reset gradients
            loss.backward()        # Compute gradients
            optimizer.step()       # Update weights

            running_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

In [ ]:
input_size = 16  # Length of one sequence
# Initialize model
hidden_size = 24  # Example, adjust as necessary
output_size = 2   # Binary classification

# mlp_model = MLPModel(max_length, hidden_size, output_size)
# mlp_model.to(device)  # Move model to device (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mlp_model = MLPModel(max_length, hidden_size, output_size).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mlp_model.parameters())

In [ ]:
train(mlp_model, optimizer, train_loader, num_epochs=25)

Epoch 1, Loss: 0.2710
Epoch 2, Loss: 0.1606
Epoch 3, Loss: 0.1090
Epoch 4, Loss: 0.0786
Epoch 5, Loss: 0.0581
Epoch 6, Loss: 0.0460
Epoch 7, Loss: 0.0358
Epoch 8, Loss: 0.0293
Epoch 9, Loss: 0.0234
Epoch 10, Loss: 0.0208
Epoch 11, Loss: 0.0172
Epoch 12, Loss: 0.0146
Epoch 13, Loss: 0.0125
Epoch 14, Loss: 0.0109
Epoch 15, Loss: 0.0097
Epoch 16, Loss: 0.0086
Epoch 17, Loss: 0.0076
Epoch 18, Loss: 0.0068
Epoch 19, Loss: 0.0060
Epoch 20, Loss: 0.0054
Epoch 21, Loss: 0.0049
Epoch 22, Loss: 0.0044
Epoch 23, Loss: 0.0040
Epoch 24, Loss: 0.0037
Epoch 25, Loss: 0.0034


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

def evaluate(model, test_loader):
    # Set model to evaluation mode
    model.eval()

    y_true = []
    y_pred = []

    with torch.no_grad():  # No gradient computation needed for inference
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            # TODO: Compute model outputs, and convert them to predicted classes
            outputs = model(images)
            _, predicted = torch.max(outputs, 1) # Predicted class, in {0, 1, ..., 9}

            # Store true and predicted labels
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    # Compute accuracy
    print(f"Test Accuracy: {accuracy_score(y_true, y_pred):.2f}%")

    # Print classification report
    print("\nClassification Report:\n")
    print(classification_report(y_true, y_pred, digits=3))

evaluate(mlp_model, test_loader)

Test Accuracy: 0.91%

Classification Report:

              precision    recall  f1-score   support

           0      0.899     0.913     0.906       195
           1      0.916     0.902     0.909       205

    accuracy                          0.907       400
   macro avg      0.907     0.908     0.907       400
weighted avg      0.908     0.907     0.908       400



In [ ]:
class ImprovedMLPModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_sizes=[64, 36, 24]):
        super(ImprovedMLPModel, self).__init__()
        self.layers = nn.ModuleList()
        prev_size = input_size
        for hidden_size in hidden_sizes:
            self.layers.append(nn.Linear(prev_size, hidden_size))
            self.layers.append(nn.BatchNorm1d(hidden_size))
            prev_size = hidden_size
        self.output_layer = nn.Linear(prev_size, output_size)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        for i, layer in enumerate(self.layers):
            if isinstance(layer, nn.Linear):
                x = layer(x)
            elif isinstance(layer, nn.BatchNorm1d):
                x = F.relu(layer(x))
                x = self.dropout(x)
        x = self.output_layer(x)
        return x

In [ ]:
# Initialize model and optimizer
mlp1_model = ImprovedMLPModel(max_length, 2)  # Adjust hidden size as needed
#mlp_model = ImprovedMLPModel(max_length, 2, dropout_rates=[0.5, 0.3, 0.1])
mlp1_model.to(device)
optimizer = torch.optim.Adam(mlp1_model.parameters())  # Adjust learning rate as necessary
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)  # Learning rate scheduler

# Training loop with learning rate scheduler
num_epochs = 50  # Increase the number of epochs

train(mlp1_model, optimizer, train_loader, num_epochs=50)

Epoch 1, Loss: 0.7016
Epoch 2, Loss: 0.6537
Epoch 3, Loss: 0.6076
Epoch 4, Loss: 0.5735
Epoch 5, Loss: 0.5284
Epoch 6, Loss: 0.4948
Epoch 7, Loss: 0.4521
Epoch 8, Loss: 0.4491
Epoch 9, Loss: 0.4182
Epoch 10, Loss: 0.3734
Epoch 11, Loss: 0.3536
Epoch 12, Loss: 0.3140
Epoch 13, Loss: 0.3310
Epoch 14, Loss: 0.2977
Epoch 15, Loss: 0.2775
Epoch 16, Loss: 0.2800
Epoch 17, Loss: 0.2608
Epoch 18, Loss: 0.2732
Epoch 19, Loss: 0.2470
Epoch 20, Loss: 0.2005
Epoch 21, Loss: 0.1966
Epoch 22, Loss: 0.2078
Epoch 23, Loss: 0.1757
Epoch 24, Loss: 0.2019
Epoch 25, Loss: 0.1884
Epoch 26, Loss: 0.1953
Epoch 27, Loss: 0.1685
Epoch 28, Loss: 0.1764
Epoch 29, Loss: 0.1578
Epoch 30, Loss: 0.1357
Epoch 31, Loss: 0.2005
Epoch 32, Loss: 0.1449
Epoch 33, Loss: 0.1696
Epoch 34, Loss: 0.1368
Epoch 35, Loss: 0.1779
Epoch 36, Loss: 0.1431
Epoch 37, Loss: 0.1567
Epoch 38, Loss: 0.1429
Epoch 39, Loss: 0.1296
Epoch 40, Loss: 0.1716
Epoch 41, Loss: 0.1167
Epoch 42, Loss: 0.1513
Epoch 43, Loss: 0.1311
Epoch 44, Loss: 0.08

In [ ]:
from sklearn.metrics import classification_report, accuracy_score



def evaluate(model, test_loader):
    # Set model to evaluation mode
    model.eval()

    y_true = []
    y_pred = []

    with torch.no_grad():  # No gradient computation needed for inference
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            # TODO: Compute model outputs, and convert them to predicted classes
            outputs = model(images)
            _, predicted = torch.max(outputs, 1) # Predicted class, in {0, 1, ..., 9}

            # Store true and predicted labels
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    # Compute accuracy
    print(f"Test Accuracy: {accuracy_score(y_true, y_pred):.2f}%")

    # Print classification report
    print("\nClassification Report:\n")
    print(classification_report(y_true, y_pred, digits=3))

evaluate(mlp1_model, test_loader)

Test Accuracy: 0.62%

Classification Report:

              precision    recall  f1-score   support

           0      0.593     0.687     0.637       195
           1      0.649     0.551     0.596       205

    accuracy                          0.618       400
   macro avg      0.621     0.619     0.616       400
weighted avg      0.622     0.618     0.616       400



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class ImprovedMLPModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(ImprovedMLPModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)  # Increase hidden size
        self.bn1 = nn.BatchNorm1d(64)
        self.fc2 = nn.Linear(64, 36)
        self.bn2 = nn.BatchNorm1d(36)
        self.fc3 = nn.Linear(36, 24)
        self.bn3 = nn.BatchNorm1d(24)
        self.fc4 = nn.Linear(24, output_size)  # Output layer
        self.dropout = nn.Dropout(0.3)  # Adjusted dropout rate
        self.leaky_relu = nn.LeakyReLU(0.01)  # Leaky ReLU

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.leaky_relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.leaky_relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.leaky_relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x)  # No activation here for regression
        return x



In [ ]:
import torch.nn as nn

# ... (rest of your imports and code) ...

def train(model, optimizer, train_loader, num_epochs=5):
    # Change the loss function to CrossEntropyLoss for classification
    criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0

        for sequences, labels in train_loader:
            # Transfer data to device
            sequences, labels = sequences.to(device), labels.to(device)

            # Forward pass
            outputs = model(sequences)
            # Calculate loss with CrossEntropyLoss
            loss = criterion(outputs, labels)

            # Backpropagation
            optimizer.zero_grad()  # Reset gradients
            loss.backward()        # Compute gradients
            optimizer.step()       # Update weights

            running_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

# ... (rest of your code) ...

In [ ]:
# Example usage (for regression):

# Initialize model and optimizer
mlp2_model = ImprovedMLPModel(max_length, 2)  # Adjust hidden size as needed
#mlp_model = ImprovedMLPModel(max_length, 2, dropout_rates=[0.5, 0.3, 0.1])
mlp2_model.to(device)
#optimizer = torch.optim.Adam(mlp_model.parameters())  # Adjust learning rate as necessary
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)  # Learning rate scheduler

# Example training loop (for regression):
learning_rate = 0.001
weight_decay = 1e-5  # L2 regularization strength

optimizer = optim.Adam(mlp2_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
#criterion = nn.MSELoss()  # Mean Squared Error for regression

# Training loop with learning rate scheduler
num_epochs = 50  # Increase the number of epochs

train(mlp2_model, optimizer, train_loader, num_epochs=50)



Epoch 1, Loss: 0.7092
Epoch 2, Loss: 0.6478
Epoch 3, Loss: 0.6081
Epoch 4, Loss: 0.5590
Epoch 5, Loss: 0.5265
Epoch 6, Loss: 0.4750
Epoch 7, Loss: 0.4600
Epoch 8, Loss: 0.4429
Epoch 9, Loss: 0.3993
Epoch 10, Loss: 0.3710
Epoch 11, Loss: 0.3769
Epoch 12, Loss: 0.3225
Epoch 13, Loss: 0.3367
Epoch 14, Loss: 0.2985
Epoch 15, Loss: 0.2982
Epoch 16, Loss: 0.2723
Epoch 17, Loss: 0.2890
Epoch 18, Loss: 0.2631
Epoch 19, Loss: 0.2469
Epoch 20, Loss: 0.2356
Epoch 21, Loss: 0.2288
Epoch 22, Loss: 0.2024
Epoch 23, Loss: 0.2004
Epoch 24, Loss: 0.1898
Epoch 25, Loss: 0.2003
Epoch 26, Loss: 0.1803
Epoch 27, Loss: 0.1809
Epoch 28, Loss: 0.1679
Epoch 29, Loss: 0.1660
Epoch 30, Loss: 0.1766
Epoch 31, Loss: 0.1506
Epoch 32, Loss: 0.1810
Epoch 33, Loss: 0.1634
Epoch 34, Loss: 0.1472
Epoch 35, Loss: 0.1508
Epoch 36, Loss: 0.1414
Epoch 37, Loss: 0.1492
Epoch 38, Loss: 0.1307
Epoch 39, Loss: 0.1316
Epoch 40, Loss: 0.1263
Epoch 41, Loss: 0.1181
Epoch 42, Loss: 0.1258
Epoch 43, Loss: 0.1498
Epoch 44, Loss: 0.12

In [ ]:
from sklearn.metrics import classification_report, accuracy_score



def evaluate(model, test_loader):
    # Set model to evaluation mode
    model.eval()

    y_true = []
    y_pred = []

    with torch.no_grad():  # No gradient computation needed for inference
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            # TODO: Compute model outputs, and convert them to predicted classes
            outputs = model(images)
            _, predicted = torch.max(outputs, 1) # Predicted class, in {0, 1, ..., 9}

            # Store true and predicted labels
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    # Compute accuracy
    print(f"Test Accuracy: {accuracy_score(y_true, y_pred):.2f}%")

    # Print classification report
    print("\nClassification Report:\n")
    print(classification_report(y_true, y_pred, digits=3))

evaluate(mlp2_model, test_loader)

Test Accuracy: 0.64%

Classification Report:

              precision    recall  f1-score   support

           0      0.620     0.677     0.647       195
           1      0.663     0.605     0.633       205

    accuracy                          0.640       400
   macro avg      0.641     0.641     0.640       400
weighted avg      0.642     0.640     0.640       400

